In [25]:
from pprint import pprint
from collections import defaultdict, OrderedDict
from random import choice, seed, randint
from tqdm import tqdm
from itertools import repeat, product, combinations, cycle
from pathlib import Path
from string import ascii_lowercase

import numpy as np
import pandas as pd

from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.datasets import ImageFolder
from torch import (load, amax as pt_amax, max as pt_max, ones, save, no_grad, stack, numel, tensor, 
                   manual_seed, sigmoid, tanh, add, mul, sub, div, amin as pt_amin, cat,
                  maximum, minimum, device, cuda, rand, prod, median, log as pt_log, round as pt_round,
                  isnan, flatten, mean)
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam
#from torchviz import make_dot
import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from deap import creator as ga_cr, base as ga_b, algorithms as ga_algo, tools as ga_t

In [26]:
#np.set_printoptions(suppress=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [27]:
device = device("cuda" if cuda.is_available() else "cpu")
# device = "cpu"
print("==> Device:", device)

==> Device: cuda


In [28]:
# Data
#X_train, y_train = load('/kaggle/input/pytorch-mnist/training.pt')

# This code cut and pasted inside eval fun

#X_test, y_test = load('/kaggle/input/pytorch-mnist/test.pt')
#X_train.shape, y_train.shape, X_test.shape, y_test.shape
#num = randint(1, len(img_F))
#img = img_F[num][0]
#imshow(img)
#img.shape

In [47]:
class NET(nn.Module):
    def __init__(self, 
                 batch_size,
                 params={
                     'image_r_crop': 64,
                     #
                    'feature_maps': 2, 
                    'f_kernel_size': 5, 
                    'f_pad_size': 2,
                    'pools': 'MaxPool2d', 
                    'p_kernel_size': 3, 
                    'f_activs': 'ELU',
                     #
                    'f_kernel_size_h': 5, 
                    'f_pad_size_h': 2,
                    'pools_h': 'MaxPool2d', 
                    'p_kernel_size_h': 3, 
                    'f_activs_h': 'ELU',
                    'times_hidden': 2,
                 },
                 outputs_dim=5,
                ):
        super().__init__()
        
        self.batch_size = batch_size
        self.image_r_crop = params['image_r_crop']
        
        # inputs model
        self.f_maps = params['feature_maps']
        self.f_k_size = params['f_kernel_size']
        self.f_pad_size = params['f_pad_size']
        self.pool = params['pools']
        self.p_k_size = params['p_kernel_size']
        self.f_activs = params['f_activs']
        
        # hidden model
        self.f_k_size_h = params['f_kernel_size_h']
        self.f_pad_size_h = params['f_pad_size_h']
        self.pool_h = params['pools_h']
        self.p_k_size_h = params['p_kernel_size_h']
        self.f_activs_h = params['f_activs_h']
        self.times_hidden = params['times_hidden']
        
        # outputs model
        self.outputs_dim = outputs_dim  
        
        self.inputs_model = nn.Sequential(
            nn.Conv2d(3, 
                      self.f_maps, 
                      self.f_k_size, 
                      padding=(self.f_pad_size, self.f_pad_size),
                      padding_mode='reflect'),
            nn.Dropout2d(p=0.25),
            nn.__getattribute__(self.f_activs)(),
            nn.__getattribute__(self.pool)(kernel_size=self.p_k_size, 
                                           stride=1))
        
        self.hidden_model = nn.Sequential(
            nn.Conv2d(self.f_maps, 
                      self.f_maps, 
                      self.f_k_size_h, 
                      padding=(self.f_pad_size_h, self.f_pad_size_h),
                      padding_mode='reflect'),
            nn.Dropout2d(p=0.25),
            nn.__getattribute__(self.f_activs_h)(),
            nn.__getattribute__(self.pool_h)(kernel_size=self.p_k_size_h, 
                                             stride=1))
        
        self._hidden_to_output_helper()
                
        self.output_model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.hidden_model_out.shape[1], self.outputs_dim),
            nn.Softmax(dim=-1)
        )
        
    def _hidden_to_output_helper(self):
        self.hidden_model_out = self.inputs_model(ones(self.batch_size,3,self.image_r_crop,self.image_r_crop))
        for _ in range(self.times_hidden):
            self.hidden_model_out = self.hidden_model(self.hidden_model_out)
        self.hidden_model_out = nn.Flatten()(self.hidden_model_out)
        
    def forward(self, x):
        """Forward"""
        res = self.inputs_model(x)
        for _ in range(self.times_hidden):
            res = self.hidden_model(res)
        res = self.output_model(res)
        return res
    
    def count_weights_biases(self):
        return int(sum(p.numel() for p in self.parameters() if p.requires_grad))
    
#net = NET(batch_size=16).train()
#print(f'forward: {net(ones(16, 3, 64, 64).float()).shape}')

In [48]:
class GA_Pytorch():
    def __init__(self, 
                 params, 
                 eval_func,
                 eval_weights,
                 #X_train,
                 #X_test,
                 #y_train,
                 #y_test,
                 #lr=0.0001,
                 sel_tournsize=2, 
                 cx_uniform_prob=0.5, 
                 mut_shuffle_idx_prob=0.1, 
                 n_pop=25, 
                 n_gen=25, 
                 n_hof=5, 
                 cx_prob=0.5, 
                 mut_prob=0.1, 
                 n_jobs=1
                ):
        self.params = params
        self.eval_func = eval_func
        self.eval_weights = eval_weights
        
        #self.X_train = X_train
        #self.X_test = X_test
        #self.y_train = y_train
        #self.y_test = y_test
        #self.image_folder = image_folder
        #self.batch_size = batch_size
        #self.lr = lr
        
        self.sel_tournsize = sel_tournsize
        self.cx_uniform_prob = cx_uniform_prob
        self.mut_shuffle_idx_prob = mut_shuffle_idx_prob
        self.n_pop = n_pop
        self.n_gen = n_gen
        self.n_hof = n_hof
        self.cx_prob = cx_prob
        self.mut_prob = mut_prob
        
        self.n_jobs = n_jobs

        self._pad_params()
        self._create_fitness_and_indiv()
        self._register_indiv_and_pop_generators()
        self._register_eval_func()
        self._register_selection_crossover_mutation_methods()

    def _pad_params(self):
        """Pad params for crossover shuffle idx method"""
        assert isinstance(self.params, dict), 'Params must be a dict, i.e. estimator.get_params()'
        params_count = {k: len(v) for k,v in self.params.items()}
        max_length, max_key = -99, ''
        for k, v in params_count.items():
            if v <= max_length:
                continue
            else:
                max_key = k
                max_length = v
        assert isinstance(max_length, int), 'The max length between all params must be an int'
        # cycle through params for max length param, otherwise infinite cycle
        values_padded = (cycle(v) if k!=max_key else v for k,v in self.params.items())
        values_padded = zip(*values_padded)  # ('a', 1, 14), ('b', 2, 16), ('c', 3, 16) ...
        values_padded = zip(*values_padded)  # ('a', 'b', 'c'), (1, 2, 3), (14, 15, 16)...
        padded_params = {}
        for k, v in zip(self.params, values_padded):
            padded_params[k] = v
        self.padded_params = padded_params
        print('Params padded')

    def _create_fitness_and_indiv(self):
        """Create GA individual and fitness entities (classes)"""
        ga_cr.create('Fitness', ga_b.Fitness, weights=self.eval_weights)
        ga_cr.create('Individual', list, fitness=ga_cr.Fitness)
        print('GA entities created')

    def _gen_params_to_ga(self):
        """Generate index for each param for individual"""
        max_dict = len(self.padded_params)
        max_length = len(list(self.padded_params.values())[0])
        idxs = [randint(0, max_length-1) for _ in range(max_dict)]
        return idxs
    
    def _register_indiv_and_pop_generators(self):
        """Register GA individual and population generators"""
        self.tb = ga_b.Toolbox()

        if self.n_jobs > 1:
            from multiprocessing import Pool
            pool = Pool()
            self.tb.register("map", pool.map)

        self.tb.register("individual", ga_t.initIterate, ga_cr.Individual, self._gen_params_to_ga)
        #print('indiv', self.tb.individual())
        self.tb.register("population", ga_t.initRepeat, list, self.tb.individual)
        #print('population', self.tb.population(n=2))
        print('GA entities\' methods registered')
        
    def _register_eval_func(self):
        """Set GA evaluate individual function"""
        self.tb.register("evaluate",
                        self.eval_func,
                        padded_params=self.padded_params,
                        #image_folder=self.image_folder,
                        #X_train=self.X_train,
                        #X_test=self.X_test, 
                        #y_train=self.y_train, 
                        #y_test=self.y_test,
                        #batch_size=self.batch_size,
                        #lr=self.lr
                        )
        #print(list(self.tb.evaluate(indiv) for indiv in self.tb.population(3)))
        print('GA eval function registered')
    
    def _register_selection_crossover_mutation_methods(self):
        self.tb.register("select", ga_t.selTournament, tournsize=self.sel_tournsize)
        self.tb.register("mate", ga_t.cxUniform, indpb=self.cx_uniform_prob)
        self.tb.register("mutate", ga_t.mutShuffleIndexes, indpb=self.mut_shuffle_idx_prob)
        print('GA sel-cx-mut methods registered')
        
    def run_ga_search(self):
        """GA Search"""
        pop = self.tb.population(n=self.n_pop)
        hof = ga_t.HallOfFame(self.n_hof)

        # Stats stdout
        #stats = ga_t.Statistics(lambda ind: ind.fitness.values )
        stats1 = ga_t.Statistics(lambda ind: ind.fitness.values[0] )
        stats2 = ga_t.Statistics(lambda ind: ind.fitness.values[1] )
        stats3 = ga_t.Statistics(lambda ind: ind.fitness.values[2] )
        stats = ga_t.MultiStatistics(accuracy=stats1, risk=stats2, complexity=stats3)
        stats.register("avg", np.mean)
        #stats.register("std", np.std)
        #stats.register("min", np.min)
        #stats.register("max", np.max)

        # History
        #hist = tools.History()
        #toolbox.decorate("select", hist.decorator)
        #tb.decorate("mate", hist.decorator)
        #tb.decorate("mutate", hist.decorator)
        #hist.update(pop)

        # GA Run
        pop, log = ga_algo.eaSimple(pop, self.tb, cxpb=self.cx_prob, 
                                    mutpb=self.mut_prob, ngen=self.n_gen, 
                                    stats=stats, halloffame=hof, verbose=True)
        
        # Convert back params
        hof_ = {}
        for i in range(self.n_hof):
            hof_['hof_' + str(i)] = self._ga_to_params(hof[i])

        return pop, log, hof_
    
    def _ga_to_params(self, idx_params):
        """Convert back idx to params"""
        res = {}
        for (k,v), idx in zip(self.padded_params.items(), idx_params):
            res[k] = v[idx]
        return res

In [49]:
net_params = {
    #image folder
    'image_resize': np.linspace(64, 128, 5).astype(int),
    'image_r_crop': np.linspace(16, 64, 5).astype(int),
    
    # inputs model
    'feature_maps': np.linspace(5, 15, 5).astype(int),
    'f_kernel_size': np.linspace(1, 5, 5).astype(int),
    'f_pad_size': np.linspace(0, 4, 5).astype(int),
    'pools': ['MaxPool2d', 'AvgPool2d'],
    'p_kernel_size': np.linspace(1, 5, 5).astype(int),
    'f_activs': ['ReLU', 'CELU', 'SELU', 'ELU', 'Softsign'],
    
    # hidden model
    'f_kernel_size_h': np.linspace(1, 5, 5).astype(int),
    'f_pad_size_h': np.linspace(0, 4, 5).astype(int),
    'pools_h': ['MaxPool2d', 'AvgPool2d'],
    'p_kernel_size_h': np.linspace(1, 5, 5).astype(int),
    'f_activs_h': ['ReLU', 'CELU', 'SELU', 'ELU', 'Softsign'],
    'times_hidden': np.linspace(1, 2, 2).astype(int),
    
    # learning
    'batch_size': np.linspace(8, 64, 5).astype(int).tolist(),
    'lr': np.linspace(0.0001, 0.01, 5),
}

def net_eval_indiv(individual, padded_params):
    """Evaluate individual's genes (estimator's params)"""
    # Params
    indiv_params = {k : list(v)[idx] for (k,v), idx in zip(padded_params.items(), individual)}
    image_folder_params = {k:v for k,v in indiv_params.items() if k in ['image_resize', 'image_r_crop']}
    net_params = {k:v for k,v in indiv_params.items() if k in [
        'feature_maps', 'f_kernel_size', 'f_pad_size', 'pools', 'p_kernel_size',
        'f_activs', 'f_kernel_size_h', 'f_pad_size_h', 'pools_h', 
        'p_kernel_size_h', 'f_activs_h', 'times_hidden', 'image_r_crop'
    ]}
    learning_params = {k:v for k,v in indiv_params.items() if k in ['batch_size', 'lr']}    
    
    # Data
    trans = transforms.Compose([
                transforms.Resize(size=(image_folder_params['image_resize'],image_folder_params['image_resize'])), 
                # transforms.RandomRotation(degrees=[-5, 5], ),
                transforms.RandomCrop(size=(image_folder_params['image_r_crop'],image_folder_params['image_r_crop'])),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    image_folder = ImageFolder('/kaggle/input/flower-photos-by-the-tensorflow-team/flower_photos', 
                        transform=trans)

    # Net
    try:
        net = NET(batch_size=learning_params['batch_size'], params=net_params)
        net = net.to(device)

        # Optimizer
        optimizer = Adam(net.parameters(), lr=learning_params['lr'])
        criterion = nn.NLLLoss()

        # Train
        #train_ds = DS(X_train, y_train)  # TODO refactor out
        train_dl = DataLoader(image_folder,
                            batch_size=learning_params['batch_size'],
                            shuffle=True,
                            num_workers=2,
                            drop_last=True)

        for epoch in range(1):
            #running_loss = []
            train_correct = 0
            train_total = 0
            for i, (inputs, labels) in enumerate(train_dl):
                if i <= 40:
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    outputs = net(inputs)
                    outputs = pt_log(outputs)

                    optimizer.zero_grad()
                    loss = criterion(outputs, labels).mean()
                    loss.backward()
                    optimizer.step()

                    # print statistics
                    #running_loss.append(loss.item())
                    _, predicted = pt_max(outputs.data, 1)
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()
                    train_accuracy = train_correct / train_total * 100
                    #print(f'TRAIN {train_accuracy:^5.2f} %', end=' ')
                else:
                    break
    except BaseException as e:
        print(e)
        return (0.01, 10, 1000000,)
        
    # Eval
    """with no_grad():
        net = net.eval()
        test_ds = DS(X_test, y_test)  # TODO refactor out
        test_dl = DataLoader(test_ds,
                            batch_size=batch_size,
                            shuffle=True,
                            drop_last=True)
        #running_loss = []
        test_correct = 0
        test_total = 0
        for i, (inputs, labels) in enumerate(test_dl):
            if i <= 100:
                #inputs = inputs.cuda()
                #labels = labels.cuda()
                outputs = net(inputs)

                # print statistics
                #running_loss.append(loss.item())
                _, predicted = pt_max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
                test_accuracy = test_correct / test_total * 100
            else:
                break
        #print(f'TEST {test_accuracy:^5.2f} %')"""
        
    # Risk
    inputs = inputs.to(device)
    risk = mean(prod(net(inputs)*10, dim=1))
    if isnan(risk):
        risk = 10
    else:
        risk = float(risk)
        
    # Complexity
    compl = net.count_weights_biases()

    return (train_accuracy, risk, compl,)

net_weights = (1, -1, -1)

In [50]:
net_ga_params = GA_Pytorch(net_params,
                           net_eval_indiv,
                           net_weights)
pop, log, hof = net_ga_params.run_ga_search()

Params padded
GA entities created
GA entities' methods registered
GA eval function registered
GA sel-cx-mut methods registered
   	      	       accuracy       	      complexity      	         risk         
   	      	----------------------	----------------------	----------------------
gen	nevals	avg   	gen	nevals	avg   	gen	nevals	avg    	gen	nevals
0  	25    	24.326	0  	25    	138754	0  	25    	14.4823	0  	25    
1  	12    	25.4722	1  	12    	115861	1  	12    	15.5279	1  	12    
2  	13    	27.8746	2  	13    	121879	2  	13    	17.165 	2  	13    
3  	15    	28.9576	3  	15    	127933	3  	15    	18.7905	3  	15    
4  	14    	28.8365	4  	14    	134784	4  	14    	17.9158	4  	14    
5  	19    	29.8273	5  	19    	159761	5  	19    	17.8441	5  	19    
6  	12    	31.1781	6  	12    	167194	6  	12    	16.5025	6  	12    
7  	12    	31.821 	7  	12    	182082	7  	12    	15.4582	7  	12    
8  	17    	30.9241	8  	17    	186953	8  	17    	15.1668	8  	17    
9  	14    	31.8527	9  	14    	190377	9  	14  

Exception ignored in: <function _releaseLock at 0x7fbd3c1a60e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fbc6413fdd0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/opt/conda/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 921, in wait
    ready = selector.select(timeout)
  File "/opt/conda/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 















14 	20    	11.6705	14 	20    	706124	14 	20    	10.5183	14 	20    







Exception ignored in: <function _releaseLock at 0x7fbd3c1a60e0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt









15 	12    	6.95109	15 	12    	844603	15 	12    	10.4032	15 	12    






16 	6     	9.71361	16 	6     	782416	16 	6     	10.6063	16 	6     




KeyboardInterrupt: 

In [33]:
pd.DataFrame(list(hof.values()))

,resize,r_crop,feature_maps,f_kernel_size,f_pad_size,pools,p_kernel_size,f_activs,f_kernel_size_h,f_pad_size_h,pools_h,p_kernel_size_h,f_activs_h,times_hidden,batch_size,lr
0,88,48,10,1,4,MaxPool2d,1,ReLU,1,4,MaxPool2d,5,ELU,1,36,0.005050
1,96,48,7,3,2,MaxPool2d,2,ReLU,1,2,MaxPool2d,4,Softsign,1,8,0.007525
2,80,56,5,3,1,AvgPool2d,5,CELU,5,0,MaxPool2d,1,ReLU,2,50,0.000100
3,88,56,10,3,3,MaxPool2d,2,CELU,1,2,MaxPool2d,5,CELU,1,8,0.005050
4,80,56,7,4,0,AvgPool2d,4,Softsign,2,3,MaxPool2d,3,SELU,2,36,0.007525
